In [ ]:
# Install fastai
!pip install fastai -q
!pip install fastai --upgrade

In [ ]:
from fastai.tabular.all import *
import pandas as pd
import csv

In [ ]:
# Load the csv as a dataframe
path = Path(".")
df = pd.read_csv(path/'finally.csv')
# Remove the id column from R's dataframe
del df['Unnamed: 0']

In [ ]:
# Get a list of categorical columns
df_cat = list(df.columns)

df_cat = df_cat[0:6] + df_cat[8: ]
# Only age and duration are numerical, the others are categorical

In [ ]:
# Get a list of numerical columns, not counting the thing we're predicting
df_num = list(df.columns)[7:8]

In [ ]:
# Use the fast.ai tabular dataloader

dls = TabularDataLoaders.from_csv(
    path/'finally.csv', y_names = "duration",
    cat_names = df_cat, cont_names = df_num,
    procs = [Categorify, FillMissing, Normalize]
    )

learn = tabular_learner(dls, y_range = (0, 33),  metrics = accuracy)

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
to = TabularPandas(df, procs = [Categorify, FillMissing, Normalize],
                   cat_names = df_cat, cont_names = df_num,
                   y_names = 'duration',
                   splits = splits)

dls = to.dataloaders(bs = 16)

In [ ]:
learn = tabular_learner(dls, y_range = (0, 33), metrics = rmse)

In [ ]:
learn.fit_one_cycle(4, lr_max = 0.01)

epoch,train_loss,valid_loss,_rmse,time
0,13.166378,16.848173,4.104652,00:00
1,14.907114,24.666842,4.966573,00:00
2,15.477958,34.251236,5.852456,00:00
3,14.255585,32.981781,5.742977,00:00


In [ ]:
learn.predict(learn.dls.train.all_cols.iloc[0])

(   sex  city  province  ...  age2_na      age2  duration
 0  0.0   0.0       0.0  ...      1.0 -2.716655  9.437881
 
 [1 rows x 70 columns], tensor([9.4379]), tensor([9.4379]))

In [ ]:
learn.export()

In [ ]:
clas, probs

(tensor([23.0137]), tensor([23.0137]))

In [ ]:
df2 = pd.read_csv(path/'finally_again.csv')

In [ ]:
to2 = TabularPandas(df, procs = [Categorify, FillMissing, Normalize],
                    cat_names = df_cat, cont_names = df_num,
                    y_names = 'duration',
                    splits = splits)

dls2 = to2.dataloaders(bs = 16)

In [ ]:
learn = tabular_learner(dls, y_range = (0, 33), metrics = rmse)

In [ ]:
test_df = df2
test_df.drop(['duration'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [ ]:
p1 = learn.get_preds(dl = dl)

In [ ]:
p2 = p1[0].tolist()

In [ ]:
 with open('testing.csv', mode='w') as employee_file:
   employee_writer = csv.writer(employee_file, delimiter = ',',
                                quotechar = '"', quoting = csv.QUOTE_MINIMAL)
   employee_writer.writerow(p2)
print("Done")

Done
